In [13]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(credentials)
sheet1 = client.open("NBA Bets 2021-22").get_worksheet(0)

In [14]:
import pandas as pd

In [15]:
mdf = pd.DataFrame(sheet1.get_all_records())

mdf

,participant full name,date,opponent,home,spread,moneyline,total,points scored,points allowed,total points scored,lost by,spread result,moneyline result,total result
0,Milwaukee Bucks,2021-10-19,Brooklyn Nets,TRUE,0.0,100,240.5,127,104,231,-23,win,win,loss
1,Brooklyn Nets,2021-10-19,Milwaukee Bucks,FALSE,0.0,-120,240.5,104,127,231,23,loss,loss,loss
2,L.A. Lakers,2021-10-19,Golden State Warriors,TRUE,-4.5,-190,230.5,114,121,235,7,loss,loss,win
3,Golden State Warriors,2021-10-19,L.A. Lakers,FALSE,4.5,160,230.5,121,114,235,-7,win,win,win
4,Charlotte Hornets,2021-10-20,Indiana Pacers,TRUE,-2.0,-130,228.5,123,122,245,-1,loss,win,win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,Dallas Mavericks,2022-01-26,Portland Trail Blazers,FALSE,-4.0,-180,216.5,132,112,244,-20,win,win,win
1442,Denver Nuggets,2022-01-26,Brooklyn Nets,FALSE,1.5,100,227.5,124,118,242,-6,win,win,win
1443,Brooklyn Nets,2022-01-26,Denver Nuggets,TRUE,-1.5,-120,227.5,118,124,242,6,loss,loss,win
1444,Chicago Bulls,2022-01-26,Toronto Raptors,TRUE,-4.0,-180,216.5,111,105,216,-6,win,win,loss


In [16]:
mdf.dtypes


participant full name     object
date                      object
opponent                  object
home                      object
spread                   float64
moneyline                  int64
total                    float64
points scored              int64
points allowed             int64
total points scored        int64
lost by                    int64
spread result             object
moneyline result          object
total result              object
dtype: object

In [17]:
def team_func(dataframe, team):
    return dataframe.loc[dataframe['participant full name'] == team]

def opp_func(dataframe, opponent):
    return dataframe.loc[dataframe['opponent'] == opponent]

def last_games_func(dataframe, n):
    return dataframe.iloc[(-1 * n):, :]

def home_func(dataframe, home_or_not):
    return dataframe.loc[dataframe['home'] == home_or_not]

def spread_func(dataframe, spread_filt_settings: dict):
    d ={
        'exact': dataframe.loc[(dataframe['spread'] == spread_filt_settings['points'])],
        'longer': dataframe.loc[dataframe['spread'] >= spread_filt_settings['points']],
        'shorter': dataframe.loc[dataframe['spread'] <= spread_filt_settings['points']]
    }
    return d[spread_filt_settings['kind']]

def favorite_func(dataframe, favorite):
    if favorite:
        return dataframe.loc[dataframe['spread'] < 0]
    else:
        return dataframe.loc[dataframe['spread'] > 0]


In [18]:
filters = {
    'team': team_func,
    'opponent': opp_func,
    'home': home_func,
    'spread': spread_func,
    'favorite': favorite_func,
    'last n games': last_games_func
}

In [ ]:
"""
filter_these = {
    'bet': "spread result" or "moneyline result" or "total result"
    'team': str,
    'opponent': str,
    'last n games': int,
    'home': "TRUE" or "FALSE",
    'spread': {
        'kind': "exact" or "longer" or "shorter",
        'points': float
        },
    'favorite': bool
}
"""

In [29]:
filter_these = {
    'bet': 'total result',
    'team': 'Golden State Warriors',
    'last n games': 7
}

In [30]:
df = mdf

for key in filters:
    if (key in filter_these) and (filter_these[key] != ''):
        df = filters[key](df, filter_these[key])

print("Record is", len(df.loc[df[filter_these['bet']] == 'win']), "wins", len(df.loc[df[filter_these['bet']] == 'loss']), "losses", len(df.loc[df[filter_these['bet']] == 'push']), "pushes")

df

Record is 4 wins 3 losses 0 pushes


,participant full name,date,opponent,home,spread,moneyline,total,points scored,points allowed,total points scored,lost by,spread result,moneyline result,total result
1252,Golden State Warriors,2022-01-14,Chicago Bulls,FALSE,3.5,155,222.0,138,96,234,-42,win,win,win
1289,Golden State Warriors,2022-01-16,Minnesota Timberwolves,FALSE,4.5,150,215.0,99,119,218,20,loss,loss,win
1316,Golden State Warriors,2022-01-18,Detroit Pistons,TRUE,-14.5,-1500,215.5,102,86,188,-16,win,win,loss
1348,Golden State Warriors,2022-01-20,Indiana Pacers,TRUE,-10.5,-750,216.5,117,121,238,4,loss,loss,win
1370,Golden State Warriors,2022-01-21,Houston Rockets,TRUE,-11.5,-555,228.5,105,103,208,-2,loss,win,loss
1399,Golden State Warriors,2022-01-23,Utah Jazz,TRUE,-6.5,-320,221.5,94,92,186,-2,loss,win,loss
1422,Golden State Warriors,2022-01-25,Dallas Mavericks,TRUE,-3.5,-165,211.5,130,92,222,-38,win,win,win


In [31]:
df['total points scored'].mean()

213.42857142857142

In [32]:
df['total points scored'].describe()

count      7.000000
mean     213.428571
std       20.614373
min      186.000000
25%      198.000000
50%      218.000000
75%      228.000000
max      238.000000
Name: total points scored, dtype: float64